# 導入套件＆TEJ API

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy import stats
from datetime import datetime


import tejapi
tejapi.ApiConfig.api_key = 'Your Key'
tejapi.ApiConfig.ignoretz = True

# 大盤行情

In [2]:
twn = tejapi.get('TWN/APRCD',
                     coid = 'Y9997',
                     mdate = {'gte': '2008-01-01', 'lte':'2021-03-30'},
                     opts = {'columns':['mdate', 'close_d']},
                     paginate = True,
                     chinese_column_name = True)
twn

,年月日,收盤價(元)
None,,
0,2008-01-02,9845.42
1,2008-01-03,9681.17
2,2008-01-04,9724.82
3,2008-01-07,9325.32
4,2008-01-08,9419.41
...,...,...
3259,2021-03-24,30991.46
3260,2021-03-25,31046.61
3261,2021-03-26,31521.77


In [3]:
#歷年春節前最後交易日
last_day = ['2008-02-01', '2009-01-21', '2010-02-10', '2011-01-28', '2012-01-18', '2013-02-06', '2014-01-27', '2015-02-13', '2016-02-03', '2017-01-24', '2018-02-12', '2019-01-30', '2020-01-20', '2021-02-05']

In [4]:
#轉換成時間物件
last_day = [datetime.fromisoformat(i) for i in last_day]
last_day

[datetime.datetime(2008, 2, 1, 0, 0),
 datetime.datetime(2009, 1, 21, 0, 0),
 datetime.datetime(2010, 2, 10, 0, 0),
 datetime.datetime(2011, 1, 28, 0, 0),
 datetime.datetime(2012, 1, 18, 0, 0),
 datetime.datetime(2013, 2, 6, 0, 0),
 datetime.datetime(2014, 1, 27, 0, 0),
 datetime.datetime(2015, 2, 13, 0, 0),
 datetime.datetime(2016, 2, 3, 0, 0),
 datetime.datetime(2017, 1, 24, 0, 0),
 datetime.datetime(2018, 2, 12, 0, 0),
 datetime.datetime(2019, 1, 30, 0, 0),
 datetime.datetime(2020, 1, 20, 0, 0),
 datetime.datetime(2021, 2, 5, 0, 0)]

In [5]:
date = []

for i in last_day:
    date.append(int(twn[twn['年月日'] == i].index.values))
    
date

[22, 263, 528, 770, 1010, 1273, 1511, 1771, 2009, 2245, 2505, 2743, 2977, 3234]

In [6]:
market = pd.DataFrame(columns = ['前十五日累計報酬', '前五日累計報酬', '封關日報酬', '後五日累計報酬', '後十五日累計報酬' ])

for i in date:
    
    # 封關前十五日累計報酬
    market.loc[i, '前十五日累計報酬'] = ((twn.loc[i, '收盤價(元)'] / twn.loc[i-15, '收盤價(元)']) - 1) * 100
    
    # 封關前五日累計報酬
    market.loc[i, '前五日累計報酬'] = ((twn.loc[i, '收盤價(元)'] / twn.loc[i-5, '收盤價(元)']) - 1) * 100
    
    # 封關當天報酬
    market.loc[i, '封關日報酬'] = ((twn.loc[i, '收盤價(元)'] / twn.loc[i-1, '收盤價(元)']) - 1) * 100
    
    # 封關後五日累計報酬
    market.loc[i, '後五日累計報酬'] = ((twn.loc[i+5, '收盤價(元)'] / twn.loc[i, '收盤價(元)']) - 1) * 100
    
    # 封關後十五日累計報酬
    market.loc[i, '後十五日累計報酬'] = ((twn.loc[i+15, '收盤價(元)'] / twn.loc[i, '收盤價(元)']) - 1) * 100

In [7]:
market = market.reset_index().drop(columns = 'index')
market

,前十五日累計報酬,前五日累計報酬,封關日報酬,後五日累計報酬,後十五日累計報酬
0,-4.425271,-0.847602,2.032412,2.826616,10.374282
1,-7.476115,-1.684823,0.12638,5.256177,4.448289
2,-8.440425,-1.328583,1.097643,-0.077208,4.118403
3,4.128801,2.132643,0.472599,-5.028556,-4.568379
4,2.508545,0.632705,0.174673,6.100606,8.792279
5,2.67811,0.940427,0.249842,0.519501,1.372118
6,-0.440498,-1.844076,-1.578669,-0.378223,1.335899
7,0.618446,0.775461,0.349648,0.80023,0.104198
8,4.134476,1.997484,-0.83916,3.249893,7.398542
9,1.887996,0.998717,0.253646,1.008615,3.344459


In [8]:
data = pd.DataFrame(columns= ['平均漲幅', '上漲次數', '上漲機率', '上漲平均漲幅', '下跌次數', '下跌機率', '下跌平均跌幅'],
                   index = ['前15日', '前5日', '封關當日', '後5日', '後15日'])

In [9]:
for i in range(0,5):
    data['平均漲幅'][i] = market.iloc[:, i].mean()
    
    pos = list(filter(lambda x: (x > 0), market.iloc[:,i]))
    data['上漲次數'][i] = len(pos)
    data['上漲機率'][i] = len(pos) / len(last_day)
    data['上漲平均漲幅'][i] = np.mean(pos)
    
    neg = list(filter(lambda x: (x < 0), market.iloc[:,i]))    
    data['下跌次數'][i] = len(neg)
    data['下跌機率'][i] = len(neg) / len(last_day)
    data['下跌平均跌幅'][i] = np.mean(neg)

In [10]:
data

,平均漲幅,上漲次數,上漲機率,上漲平均漲幅,下跌次數,下跌機率,下跌平均跌幅
前15日,-0.616721,9,0.642857,2.151487,5,0.357143,-5.599495
前5日,0.162666,9,0.642857,1.420003,5,0.357143,-2.100538
封關當日,0.262072,12,0.857143,0.507237,2,0.142857,-1.208915
後5日,1.353908,10,0.714286,2.893661,4,0.285714,-2.495473
後15日,3.151834,12,0.857143,4.305301,2,0.142857,-3.768963


# 事件研究個股異常報酬

In [11]:
security = tejapi.get('TWN/ANPRCSTD',
                     mkt = 'TSE',
                     stypenm = '普通股',
                     opts = {'columns':['coid','mdate','stypenm','mkt']},
                     paginate = True,
                     chinese_column_name = True)
tse_stocks = security['證券碼'].tolist()

## 封關前

In [12]:
#計算約3~4hr
final = pd.DataFrame()
for stock in tse_stocks:
    for date in last_day:
        return_set = tejapi.get('TWN/APRCD',
                                coid = stock,
                                mdate = {'gte': date + pd.Timedelta(days = -450), 'lte': date + pd.Timedelta(days = 25)}, #確保有涵蓋到-260期~5期交易日 
                                opts = {'columns' : ['coid', 'mdate','roi']},
                                paginate = True,
                                chinese_column_name = True)
        
        #跳過無充足資料，並印出提示
        if len(return_set[return_set['年月日'] == date]) == 0: 
            print(stock +' has no ' + str(date) +' event day return available')
            continue
            
        #觀察執行到哪間公司
        print(stock, date)
        
        #封關日的 index, 做為切割點                  
        event_index = return_set[return_set['年月日'] == date].index.values.astype(int)[0]
        
        #-260~-10期的報酬(共251期)
        estimate_window = return_set[event_index - 260 - 1: event_index - 10].reset_index(drop=True)
        
        #-5~+5期的報酬(共11期) 
        event_window = return_set[event_index - 5: event_index + 5 + 1].reset_index(drop=True) 
        
        #避免無完整股價資料
        if len(estimate_window) < 251 or len(event_window) < 11:
            print(stock +' has no complete ' + str(date) +' window return available')
            print('jump', stock, date)
            continue
            
        #估計起始日期(-260)
        estimate_start = estimate_window.loc[0, '年月日']
        #事件結束日期(+5)  
        event_end = event_window.loc[5*2, '年月日']  
        
        #先用fama french 五因子模型
        fama_french = tejapi.get('TWN/AFACTO1D',
                            coid = 'Y9999',
                            mdate = {'gte':estimate_start, 'lte': event_end}, #直接抓-260~+5  
                            opts = {'columns': ['coid', 'mdate','mrp','smbn','bp','op','inv']},
                            paginate = True,
                            chinese_column_name = True
                            )
        
        #與估計期間合併,形成OLS估計所需data
        ols_data = estimate_window.merge(fama_french, on = '年月日') 
        x = ols_data.loc[:,['市場風險溢酬', '規模溢酬(5因子)','淨值市價比溢酬','盈利能力因子','投資因子']].values
        y = ols_data['報酬率％'].values.reshape(-1,1)
        
        #線性回歸(OLS)
        model = LinearRegression()
        model.fit(x,y)
        
        #與事件期間合併
        predict_data = event_window.merge(fama_french, on = '年月日') 
        #事件期間的因子
        event_x = predict_data.loc[:,['市場風險溢酬','規模溢酬(5因子)','淨值市價比溢酬','盈利能力因子','投資因子']].values
        
        #計算相關報酬率
        event_window['預期報酬率'] = model.predict(event_x)
        event_window['異常報酬率'] = event_window['報酬率％'] - event_window['預期報酬率'] 
        event_window['累計異常報酬率'] = event_window['異常報酬率'].cumsum()
        
        #加上相對天數,利於之後算分群算平均
        event_window['相對天數'] = [i for i in range(-5, 5 + 1)]   #***
        
        #儲存
        final = final.append(event_window[['證券代碼','異常報酬率','累計異常報酬率','相對天數']]).reset_index(drop=True)
    

1101 2008-02-01 00:00:00
1101 2009-01-21 00:00:00
1101 2010-02-10 00:00:00
1101 2011-01-28 00:00:00
1101 2012-01-18 00:00:00
1101 2013-02-06 00:00:00
1101 2014-01-27 00:00:00
1101 2015-02-13 00:00:00
1101 2016-02-03 00:00:00
1101 2017-01-24 00:00:00
1101 2018-02-12 00:00:00
1101 2019-01-30 00:00:00
1101 2020-01-20 00:00:00
1101 2021-02-05 00:00:00
1102 2008-02-01 00:00:00
1102 2009-01-21 00:00:00
1102 2010-02-10 00:00:00
1102 2011-01-28 00:00:00
1102 2012-01-18 00:00:00
1102 2013-02-06 00:00:00
1102 2014-01-27 00:00:00
1102 2015-02-13 00:00:00
1102 2016-02-03 00:00:00
1102 2017-01-24 00:00:00
1102 2018-02-12 00:00:00
1102 2019-01-30 00:00:00
1102 2020-01-20 00:00:00
1102 2021-02-05 00:00:00
1103 2008-02-01 00:00:00
1103 2009-01-21 00:00:00
1103 2010-02-10 00:00:00
1103 2011-01-28 00:00:00
1103 2012-01-18 00:00:00
1103 2013-02-06 00:00:00
1103 2014-01-27 00:00:00
1103 2015-02-13 00:00:00
1103 2016-02-03 00:00:00
1103 2017-01-24 00:00:00
1103 2018-02-12 00:00:00
1103 2019-01-30 00:00:00


In [13]:
# 備份
test = final.copy()
test

,證券代碼,異常報酬率,累計異常報酬率,相對天數
0,1101,3.318230,3.318230,-5
1,1101,-3.387718,-0.069488,-4
2,1101,-2.606134,-2.675623,-3
3,1101,-0.557490,-3.233113,-2
4,1101,0.902388,-2.330724,-1
...,...,...,...,...
117299,9958,3.030720,5.012627,1
117300,9958,-1.816393,3.196235,2
117301,9958,-0.336534,2.859700,3
117302,9958,0.956537,3.816238,4


In [14]:
test

,證券代碼,異常報酬率,累計異常報酬率,相對天數
0,1101,3.318230,3.318230,-5
1,1101,-3.387718,-0.069488,-4
2,1101,-2.606134,-2.675623,-3
3,1101,-0.557490,-3.233113,-2
4,1101,0.902388,-2.330724,-1
...,...,...,...,...
117299,9958,3.030720,5.012627,1
117300,9958,-1.816393,3.196235,2
117301,9958,-0.336534,2.859700,3
117302,9958,0.956537,3.816238,4


In [15]:
#先輸出成csv檔
test.to_csv("春節.csv", index = False, line_terminator='\n') 

In [16]:
test = pd.read_csv("春節.csv")
test

,證券代碼,異常報酬率,累計異常報酬率,相對天數
0,1101,3.318230,3.318230,-5
1,1101,-3.387718,-0.069488,-4
2,1101,-2.606134,-2.675623,-3
3,1101,-0.557490,-3.233113,-2
4,1101,0.902388,-2.330724,-1
...,...,...,...,...
117299,9958,3.030720,5.012627,1
117300,9958,-1.816393,3.196235,2
117301,9958,-0.336534,2.859700,3
117302,9958,0.956537,3.816238,4


In [17]:
#存成csv時，所有值都變數值，所以要先轉格式
test['證券代碼'] = test['證券代碼'].astype(str)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117304 entries, 0 to 117303
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   證券代碼     117304 non-null  object 
 1   異常報酬率    117304 non-null  float64
 2   累計異常報酬率  117304 non-null  float64
 3   相對天數     117304 non-null  int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 3.6+ MB


## 封關前

In [63]:
pre_close = pd.DataFrame()

for stock in tse_stocks:
    
    #以封關當天累計異常報酬進行檢定 
    df = test[test['證券代碼'] == stock].reset_index(drop=True)
    df = df[df['相對天數'] == 0]
    
    #刪除樣本過少公司，不然檢定所需樣本過少
    if len(df) < 4:
        print(stock, 'has no enough data')
        continue
    
    sample = df['累計異常報酬率'].values
    
    #檢定
    t, p_value = stats.ttest_1samp(sample, 0)
    if p_value <= 0.01:
        significance = '***'
    elif 0.01 < p_value <= 0.05:
        significance = '**'
    elif 0.05 < p_value <= 0.1:
        significance = '*'
    else:
        significance = ''  
            
    pre_close = pre_close.append(pd.DataFrame(np.array([stock,t,p_value, significance]).reshape((1,4)), columns = ['證券代碼','T檢定值', 'P-value','顯著水準'],)).reset_index(drop=True)

1342 has no enough data
1587 has no enough data
1760 has no enough data
2211 has no enough data
2241 has no enough data
2247 has no enough data
2630 has no enough data
2753 has no enough data
2897 has no enough data
2945 has no enough data
3138 has no enough data
3530 has no enough data
3543 has no enough data
3592 has no enough data
3711 has no enough data
3712 has no enough data
3714 has no enough data
4155 has no enough data
4440 has no enough data
4540 has no enough data
4562 has no enough data
4564 has no enough data
4566 has no enough data
4572 has no enough data
4576 has no enough data
4764 has no enough data
4766 has no enough data
4770 has no enough data
4961 has no enough data
4967 has no enough data
4989 has no enough data
5222 has no enough data
5283 has no enough data
5876 has no enough data
6288 has no enough data
6416 has no enough data
6438 has no enough data
6491 has no enough data
6515 has no enough data
6533 has no enough data
6558 has no enough data
6579 has no enou

In [64]:
pre_close

,證券代碼,T檢定值,P-value,顯著水準
0,1101,-0.0723332618924207,0.943437729771336,
1,1102,0.6190505133652855,0.5465787856536315,
2,1103,0.741719307293419,0.4714457566629393,
3,1104,-1.4239963599350374,0.17800595830327962,
4,1108,-2.089183508540014,0.056918371948216695,*
...,...,...,...,...
809,9944,0.08988227951480475,0.9297508235454666,
810,9945,1.8678144973402493,0.08449163575674622,*
811,9946,0.32491027205085854,0.7504183822907546,
812,9955,-0.7076774300073677,0.4916388130909437,


In [65]:
pre_close['T檢定值'] = pre_close['T檢定值'].astype(float)
pre_close['P-value'] = pre_close['P-value'].astype(float)

In [66]:
#選擇正顯著
result = pre_close[(pre_close['T檢定值'] > 0) & (pre_close['P-value'] < 0.01) ].reset_index(drop=True)
result

,證券代碼,T檢定值,P-value,顯著水準
0,2022,3.028258,0.009697,***
1,2031,3.033923,0.009592,***
2,2383,3.759440,0.002384,***


In [67]:
#正顯著股票清單
positive_list = result['證券代碼'].tolist()
positive_list

['2022', '2031', '2383']

In [68]:
#平均異常報酬
for i in range(-5, 1):
    result['第' + str(i) + '天平均異常報酬'] = test[test['證券代碼'].isin(positive_list) & (test['相對天數'] == i)].groupby('證券代碼')['異常報酬率'].mean().reset_index(drop=True)
result

,證券代碼,T檢定值,P-value,顯著水準,第-5天平均異常報酬,第-4天平均異常報酬,第-3天平均異常報酬,第-2天平均異常報酬,第-1天平均異常報酬,第0天平均異常報酬
0,2022,3.028258,0.009697,***,0.469080,0.157615,0.595925,0.439735,0.548148,0.551976
1,2031,3.033923,0.009592,***,0.897434,0.487129,0.561632,0.506703,0.708076,0.804337
2,2383,3.759440,0.002384,***,0.250825,1.423494,-0.022014,0.866391,1.902860,-0.024344


In [69]:
result['期間累計平均異常報酬率'] = test[test['證券代碼'].isin(positive_list) & (test['相對天數'] == 0)].groupby('證券代碼')['累計異常報酬率'].mean().reset_index(drop=True)
result

,證券代碼,T檢定值,P-value,顯著水準,第-5天平均異常報酬,第-4天平均異常報酬,第-3天平均異常報酬,第-2天平均異常報酬,第-1天平均異常報酬,第0天平均異常報酬,期間累計平均異常報酬率
0,2022,3.028258,0.009697,***,0.469080,0.157615,0.595925,0.439735,0.548148,0.551976,2.762479
1,2031,3.033923,0.009592,***,0.897434,0.487129,0.561632,0.506703,0.708076,0.804337,3.965310
2,2383,3.759440,0.002384,***,0.250825,1.423494,-0.022014,0.866391,1.902860,-0.024344,4.397212


In [70]:
result['公司中文簡稱'] = ''
result['TEJ產業名'] = ''
result['TEJ子產業名'] = ''

for i in range(len(positive_list)):
    firm_info = tejapi.get('TWN/AIND',
                          coid = positive_list[i],
                          opts = {'columns':['coid', 'inamec', 'tejind2_c', 'tejind3_c']},
                          chinese_column_name = True,
                          paginate = True)
    result.loc[i, '公司中文簡稱'] = firm_info.loc[0, '公司中文簡稱']
    result.loc[i, 'TEJ產業名'] = firm_info.loc[0, 'TEJ產業名']
    result.loc[i, 'TEJ子產業名'] = firm_info.loc[0, 'TEJ子產業名']

In [71]:
result

,證券代碼,T檢定值,P-value,顯著水準,第-5天平均異常報酬,第-4天平均異常報酬,第-3天平均異常報酬,第-2天平均異常報酬,第-1天平均異常報酬,第0天平均異常報酬,期間累計平均異常報酬率,公司中文簡稱,TEJ產業名,TEJ子產業名
0,2022,3.028258,0.009697,***,0.469080,0.157615,0.595925,0.439735,0.548148,0.551976,2.762479,聚亨,M20A 金屬基本,M20A1 線材
1,2031,3.033923,0.009592,***,0.897434,0.487129,0.561632,0.506703,0.708076,0.804337,3.965310,新光鋼,M20A 金屬基本,M20A3 鋼板
2,2383,3.759440,0.002384,***,0.250825,1.423494,-0.022014,0.866391,1.902860,-0.024344,4.397212,台光電,M23B 主機板,M23B3A 銅箔基板


In [72]:
firm_list = result['證券代碼'].to_list()

In [73]:
firms = tejapi.get('TWN/AIND',
                  coid = firm_list,
                  opts = {'columns':['coid','fnamec','tejind4_c']},
                  paginate = True,
                  chinese_column_name = True)
firms

,公司簡稱,公司中文全稱,TSE新產業名
None,,,
0,2022,聚亨企業,M2000 鋼鐵工業
1,2031,新光鋼鐵,M2000 鋼鐵工業
2,2383,台光電子材料,M2328 電子零組件


## 封關後

In [74]:
ttest2 = pd.DataFrame()

for stock in tse_stocks:
    
    #以最後一天累計異常報酬進行檢定 
    df = test[test['證券代碼'] == stock].reset_index(drop=True)
    df = df[df['相對天數'] == 5]
    
    #刪除樣本過少公司，不然檢定所需樣本過少
    if len(df) < 4:
        print(stock, 'has no enough data')
        continue
    
    sample = df['累計異常報酬率'].values
    
    #檢定
    t, p_value = stats.ttest_1samp(sample, 0)
    if p_value <= 0.01:
        significance = '***'
    elif 0.01 < p_value <= 0.05:
        significance = '**'
    elif 0.05 < p_value <= 0.1:
        significance = '*'
    else:
        significance = ''  
            
    ttest2 = ttest2.append(pd.DataFrame(np.array([stock,t,p_value, significance]).reshape((1,4)), columns = ['證券代碼','T檢定值', 'P-value','顯著水準'],)).reset_index(drop=True)
ttest2

1342 has no enough data
1587 has no enough data
1760 has no enough data
2211 has no enough data
2241 has no enough data
2247 has no enough data
2630 has no enough data
2753 has no enough data
2897 has no enough data
2945 has no enough data
3138 has no enough data
3530 has no enough data
3543 has no enough data
3592 has no enough data
3711 has no enough data
3712 has no enough data
3714 has no enough data
4155 has no enough data
4440 has no enough data
4540 has no enough data
4562 has no enough data
4564 has no enough data
4566 has no enough data
4572 has no enough data
4576 has no enough data
4764 has no enough data
4766 has no enough data
4770 has no enough data
4961 has no enough data
4967 has no enough data
4989 has no enough data
5222 has no enough data
5283 has no enough data
5876 has no enough data
6288 has no enough data
6416 has no enough data
6438 has no enough data
6491 has no enough data
6515 has no enough data
6533 has no enough data
6558 has no enough data
6579 has no enou

,證券代碼,T檢定值,P-value,顯著水準
0,1101,0.8793663516242111,0.39516709705658415,
1,1102,0.8181673262469165,0.4280041959451526,
2,1103,0.7415378245655924,0.4715520380516778,
3,1104,-0.6455010529430157,0.5298314586836088,
4,1108,-1.9933012154280176,0.06764657117606752,*
...,...,...,...,...
809,9944,0.13457436320476995,0.8950096269570773,
810,9945,0.34149115025594323,0.7381906242065985,
811,9946,0.5837538916790092,0.5693764903592549,
812,9955,-0.1937871083240667,0.8493377669168116,


In [75]:
ttest2['T檢定值'] = ttest2['T檢定值'].astype(float)
ttest2['P-value'] = ttest2['P-value'].astype(float)

In [76]:
#選擇正顯著
result = ttest2[(ttest2['T檢定值'] > 0) & (ttest2['P-value'] < 0.01) ].reset_index(drop=True)
result

,證券代碼,T檢定值,P-value,顯著水準
0,2014,3.184702,0.007176,***
1,2029,3.068631,0.008972,***
2,2383,4.129096,0.001187,***
3,3027,3.260697,0.006199,***


In [77]:
#正顯著股票清單
positive_list = result['證券代碼'].tolist()
positive_list

['2014', '2029', '2383', '3027']

In [78]:
#平均異常報酬
for i in range(1, 5+1):
    result['第' + str(i) + '天平均異常報酬'] = test[test['證券代碼'].isin(positive_list) & (test['相對天數'] == i)].groupby('證券代碼')['異常報酬率'].mean().reset_index(drop=True)
    
result

,證券代碼,T檢定值,P-value,顯著水準,第1天平均異常報酬,第2天平均異常報酬,第3天平均異常報酬,第4天平均異常報酬,第5天平均異常報酬
0,2014,3.184702,0.007176,***,1.487091,0.726856,-0.410263,0.811542,0.154996
1,2029,3.068631,0.008972,***,0.650190,0.494653,-0.421663,0.963163,-0.042668
2,2383,4.129096,0.001187,***,1.050817,-0.436106,0.327196,-0.154444,0.249025
3,3027,3.260697,0.006199,***,1.698408,-1.042389,0.522391,1.157678,-0.264020


In [79]:
result['期間平均累計異常報酬率'] = test[test['證券代碼'].isin(positive_list) & (test['相對天數'] == 5)].groupby('證券代碼')['累計異常報酬率'].mean().reset_index(drop=True) - test[test['證券代碼'].isin(positive_list) & (test['相對天數'] == 0)].groupby('證券代碼')['累計異常報酬率'].mean().reset_index(drop=True)
result

,證券代碼,T檢定值,P-value,顯著水準,第1天平均異常報酬,第2天平均異常報酬,第3天平均異常報酬,第4天平均異常報酬,第5天平均異常報酬,期間平均累計異常報酬率
0,2014,3.184702,0.007176,***,1.487091,0.726856,-0.410263,0.811542,0.154996,2.770223
1,2029,3.068631,0.008972,***,0.650190,0.494653,-0.421663,0.963163,-0.042668,1.643675
2,2383,4.129096,0.001187,***,1.050817,-0.436106,0.327196,-0.154444,0.249025,1.036487
3,3027,3.260697,0.006199,***,1.698408,-1.042389,0.522391,1.157678,-0.264020,2.072069


In [80]:
result['公司中文簡稱'] = ''
result['TEJ產業名'] = ''
result['TEJ子產業名'] = ''

for i in range(len(positive_list)):
    firm_info = tejapi.get('TWN/AIND',
                          coid = positive_list[i],
                          opts = {'columns':['coid', 'inamec', 'tejind2_c', 'tejind3_c']},
                          chinese_column_name = True,
                          paginate = True)
    result.loc[i, '公司中文簡稱'] = firm_info.loc[0, '公司中文簡稱']
    result.loc[i, 'TEJ產業名'] = firm_info.loc[0, 'TEJ產業名']
    result.loc[i, 'TEJ子產業名'] = firm_info.loc[0, 'TEJ子產業名']

In [81]:
result

,證券代碼,T檢定值,P-value,顯著水準,第1天平均異常報酬,第2天平均異常報酬,第3天平均異常報酬,第4天平均異常報酬,第5天平均異常報酬,期間平均累計異常報酬率,公司中文簡稱,TEJ產業名,TEJ子產業名
0,2014,3.184702,0.007176,***,1.487091,0.726856,-0.410263,0.811542,0.154996,2.770223,中鴻,M20A 金屬基本,M20A3 鋼板
1,2029,3.068631,0.008972,***,0.650190,0.494653,-0.421663,0.963163,-0.042668,1.643675,盛餘,M20A 金屬基本,M20A3 鋼板
2,2383,4.129096,0.001187,***,1.050817,-0.436106,0.327196,-0.154444,0.249025,1.036487,台光電,M23B 主機板,M23B3A 銅箔基板
3,3027,3.260697,0.006199,***,1.698408,-1.042389,0.522391,1.157678,-0.264020,2.072069,盛達,M23K 通訊設備,M23KX 其他網通


In [82]:
firm_list = result['證券代碼'].to_list()

In [83]:
firms = tejapi.get('TWN/AIND',
                  coid = firm_list,
                  opts = {'columns':['coid','fnamec','tejind4_c']},
                  paginate = True,
                  chinese_column_name = True)
firms

,公司簡稱,公司中文全稱,TSE新產業名
None,,,
0,2014,中鴻鋼鐵,M2000 鋼鐵工業
1,2029,盛餘,M2000 鋼鐵工業
2,2383,台光電子材料,M2328 電子零組件
3,3027,盛達電業,M2327 通信網路業
